## circular_dilation_raw_cells
The purpose of this notebook is to figure out a good way to turn cell centers detected in raw space into more circular looking objects. Since we don't always have isotropic resolution, it should suffice to have cylindrical cells (for now), i.e the z dilation will simply be with a rectangular kernel. 

The tradeoff in doing this is between how circular something needs to appear and the number of pixels you need to do that. We don't want to use too many pixels because we will lose the ability to distinguish between neighboring cells and also will block out the other layers. On the other hand, we need to use enough pixels so that the cells actually look good enough for publication. A plus sign (technically the smallest non trivial pixelized circle) is not good enough, for example. 

In [17]:
import numpy as np
import os
import csv
import matplotlib.pyplot as plt
import neuroglancer
%matplotlib inline

In [13]:
animal_id = 21
pth=os.path.join('/jukebox/wang/Jess/lightsheet_output',
        '201904_ymaze_cfos','processed',f'an{animal_id}','clearmap_cluster_output',
        'cells.npy')

In [14]:
#first, make a map of cells
converted_points = np.load(pth)

In [15]:
for point in converted_points[0:10]:
    print(point)

[ 570 1516   51]
[ 571 1491   49]
[ 572 1509   51]
[ 577 1388   48]
[ 581 1520   49]
[ 582 1338   49]
[ 582 1392   46]
[ 583 1474   45]
[ 583 1533   51]
[ 584 1434   46]


In [9]:
len(converted_points)

243653

In [16]:
header = "Coordinate 1,Coordinate 2,Ellipsoid Dimensions,Tags,Description,Segment IDs,Parent ID,Type,ID\n"
output_filename = '/home/ahoag/ngdemo/data/cell_annotations_test.csv'
with open(output_filename,'w') as outfile:
    outfile.write(header)
    for point in converted_points[::10]:
        x,y,z=point
        outline=f'''"({x},{y},{z})",,,,,,,Point,\n'''
        outfile.write(outline)
print(f"Wrote {output_filename} ")

Wrote /home/ahoag/ngdemo/data/cell_annotations_test.csv 


In [19]:
# neuroglancer.set_static_content_source(url='https://nglancer.pni.princeton.edu')
neuroglancer.set_static_content_source(url='http://localhost:8080')
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
    s.layers['atlas'] = neuroglancer.ImageLayer(source='precomputed://http://localhost:1339'
    )
print(viewer)

http://127.0.0.1:34937/v/0de607563e3bbaef00263dabfa3a5237de16c594/


In [20]:
with viewer.txn() as s:
    

In [27]:
counter=0
with viewer.txn() as s:
    s.layers['annotation']=neuroglancer.AnnotationLayer()
    annotations=s.layers['annotation'].annotations
    
    for (x,y,z) in converted_points:
    #     print(x,y,z)
        pt=neuroglancer.PointAnnotation(point=[x, y, z],id=f'point{counter}')
        annotations.append(pt)
        counter+=1